In [1]:
from datasets import Dataset, DatasetDict
import json

c:\ProgramData\Anaconda3\envs\llm_final\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### `json` data

In [12]:
with open("../datasets/json_data/example_data.json", "r", encoding='utf-8') as f:
    data = json.load(f)
    

data

[{'question': '입영일자 연기 제도란 무엇인가요?',
  'answer': '현역병 입영 대상자로서 질병 등 부득이한 사유로 지정된 입영일자에 입영할 수 없는 사람이 병역이행일 연기신청서를 제출하면, 이를 심사하여 입영일자를 연기하는 제도입니다.'},
 {'question': '입영일자 연기 신청 대상자는 누구인가요?',
  'answer': '현역병입영 대상자로서 아래 연기사유에 해당되는 사람입니다. 단, 입영일자나 입영부대를 본인이 선택한 사람은 연기 처리 제한이 있으며, 예외적으로 질병, 직계 존·비속 간호, 천재·지변, 행방불명, 각군 지원 사유자는 연기가 가능합니다.'},
 {'question': '입영일자 연기 신청은 언제까지 가능한가요?', 'answer': '입영일 5일 전까지 신청이 가능합니다.'},
 {'question': '입영일자 연기 신청은 어디에서 하나요?',
  'answer': '지방병무청 현역입영 부서(현역입영과, 병역판정입영과, 병역관리과)에서 신청할 수 있습니다. 우편 제출 시, 병무청 「병무민원포털」에서 병역이행일 연기신청서를 다운받아 작성 후, 입영통지서를 받은 지방병무청(지청)으로 송부하면 됩니다.'}]

#### `json` -> `dataset dict` 로 변환 및 저장

In [13]:
# 변환
dataset = Dataset.from_list(data)

dataset_dict = DatasetDict({
    "train": dataset
})

In [14]:
# 저장
dataset_dict.save_to_disk('../datasets/dataset_dict_data/example_data')

Saving the dataset (1/1 shards): 100%|██████████| 4/4 [00:00<00:00, 573.36 examples/s]


In [ ]:
# rf. dataset_dict  column 이름 변경하는 법
#dataset_dict = dataset_dict.rename_column("response", "completion")

#### `dataset dict` data

In [15]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 4
    })
})

In [20]:
train = dataset_dict['train']
train

Dataset({
    features: ['question', 'answer'],
    num_rows: 4
})

In [19]:
train[0]

{'question': '입영일자 연기 제도란 무엇인가요?',
 'answer': '현역병 입영 대상자로서 질병 등 부득이한 사유로 지정된 입영일자에 입영할 수 없는 사람이 병역이행일 연기신청서를 제출하면, 이를 심사하여 입영일자를 연기하는 제도입니다.'}

### `train_data` 만들기


#### 방법1
- SFTTrainer는 특정 데이터셋 딕셔너리 형식을 전처리 없이 바로 사용할 수 있는 (자동변환을) 지원한다.
- 문제는 몇몇 모델 중 이 자동변환에 오류가 있는 경우가 있다.(EXAONE이 그러하다..)
- 대부분의 모델들에서는 잘 작동한다. 방법은 아래와 같다.

In [30]:
from datasets import load_from_disk
dataset_dict = load_from_disk('../datasets/dataset_dict_data/example_data')

In [31]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 4
    })
})

In [34]:
dataset_dict['train'][0]

{'question': '입영일자 연기 제도란 무엇인가요?',
 'answer': '현역병 입영 대상자로서 질병 등 부득이한 사유로 지정된 입영일자에 입영할 수 없는 사람이 병역이행일 연기신청서를 제출하면, 이를 심사하여 입영일자를 연기하는 제도입니다.'}

In [32]:
from datasets import DatasetDict, Dataset

system_prompt = (
    "You are a helpful assistant."
)

def transform_row(example):
    prompt = example['question']
    response = example['answer']
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response},
        
    ]
    return {"messages": messages}

new_train_dataset = dataset_dict['train'].map(transform_row, remove_columns=['question', 'answer'])

new_dataset_dict = DatasetDict({
    'train': new_train_dataset
})

In [33]:
new_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 4
    })
})

In [29]:
new_dataset_dict['train'][0]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': '입영일자 연기 제도란 무엇인가요?', 'role': 'user'},
  {'content': '현역병 입영 대상자로서 질병 등 부득이한 사유로 지정된 입영일자에 입영할 수 없는 사람이 병역이행일 연기신청서를 제출하면, 이를 심사하여 입영일자를 연기하는 제도입니다.',
   'role': 'assistant'}]}

In [35]:
new_dataset_dict.save_to_disk('../datasets/train_data/example_data')

Saving the dataset (1/1 shards): 100%|██████████| 4/4 [00:00<00:00, 950.50 examples/s] 


#### 방법 2
- SFTTrainer 에 직접 formatting_func 설정
- 아래는 예시코드이다. 

In [ ]:
# trainer = SFTTrainer(
#     .....
#     train_dataset= dataset,
#     formatting_func = formatting_prompt_func, 
#     ....
# )
# trainer.train()

In [ ]:
# train_dataset 예시
from datasets import load_from_disk
dataset_dict = load_from_disk('../datasets/dataset_dict_data/example_data')
dataset = dataset_dict['train']

In [ ]:
# formatting_func 예시
def formatting_prompt_func(example):
    output_texts = []
    system_message = "너는 병역이행법 상담을 도와주는 챗봇이야."
    for i in range(len(example['question'])):
        text = f"[|system|]{system_message}[|endofturn|]\n[|user|]{example['question'][i]}[|endofturn|]\n[|assistant|]{example['answer'][i]}[|endofturn|]"
        output_texts.append(text)
    return output_texts